This is a notebook for a simple DMRG algorithm: Modern MPS Version

The code shown in this notebook is adapted from the software library:          ITensor https://github.com/ITensor
and revised by Wenqi Yang

We take 1D Heisenberg model as an example, and show how to make good use of ITensor lib to write DMRG code. We adopt MPO (matrix product operator) approach and optimize double sites every step.

If we define sites in the following way:

            sites = siteinds("S=1/2",N)

it means that we do not make use of the U(1) symmetry in this system. 

The computational efficiency will decrease.

H is the many-body Hamiltonian in the MPO form.

psi0 is the many-body wavefunction in the MPS form.

Here, we set the initial MPS state as psi0, and do iteration to optimize the MPS state to make it get close to the ground state of the system.

In [8]:
using ITensors
using KrylovKit
let
    #define MPO and initial MPS state
    N = 100 #length of the Heisenberg chain
    sites = siteinds("S=1/2",N;conserve_qns=true)    
    os = OpSum()
    for j in 1:(N-1)
        os .+= 0.5,"S+",j,"S-",j+1
        os .+= 0.5,"S-",j,"S+",j+1
        os .+= "Sz",j,"Sz",j+1
    end
    H = MPO(os,sites)
    state = [isodd(n) ? "Up" : "Dn" for n in 1:N]
    psi0 = MPS(sites,state)
    #set DMRG parameters
    nsweep = 4
    maxdim = 10 #bond dimension cut off
    #do DMRG calculation
    PH = ProjMPO(H) #ProjMPO is a type of ITensor
    psi = copy(psi0)
    N = length(psi)
    if !isortho(psi) || orthoCenter(psi)!=1
        psi=ITensors.orthogonalize!(PH,psi,1) # set the orthogonal center at the first site
    end
    PH = position!(PH,psi,1) #Effective Hamiltonian for the first-two sites
    energy = 0.0
    for sw in 1:nsweep
        for (b,ha) in sweepnext(N)
            PH = position!(PH,psi,b) #Effective Hamiltonian for the two sites connected by the bond we want to optimaize
            phi = psi[b]*psi[b+1] # coarse-grain two sites of the MPS
            vals,vecs = eigsolve(PH,phi,1,:SR;ishermitian=true,tol=1e-14,krylovdim=3,maxiter=1) #get the ground eigenvalue and the ground eigenstate of the coarsed site
            energy = vals[1]
            phi = vecs[1]
            ortho = ha==1 ? "left" : "right" #judge the sweep direction
            spec = replacebond!(
            PH,psi,b,phi;maxdim=maxdim,ortho=ortho,normalize=true)
            # replace the two optimized site of MPS by the eigenvector we get (after factorize the eigenvector)
            # if the sweep dirction is "right" the factorize direction is "left" (get the left canonical tensor )
            # so ortho is equal to "left"
        end
        println("After sweep $sw energy=$energy")
    end
end     
            

After sweep 1 energy=-43.84934485227096
After sweep 2 energy=-44.101781096345604
After sweep 3 energy=-44.11289901698599
After sweep 4 energy=-44.11508297063795
